Competition on Kaggle: https://www.kaggle.com/c/how-good-is-your-medium-article/leaderboard

In [35]:
import os
import json
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import Ridge

In [36]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [37]:
def read_json_line(line=None):
    result = None
    try:        
        result = json.loads(line)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(str(e).split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        new_line = list(line)
        new_line[idx_to_replace] = ' '
        new_line = ''.join(new_line)     
        return read_json_line(line=new_line)
    return result

In [38]:
def extract_features_and_write(path_to_data,
                               inp_filename, is_train=True):
    ident = 0
    features = ['content', 'published', 'title', 'author']
    prefix = 'train' if is_train else 'test'
    feature_files = [open(os.path.join(path_to_data,
                                       '{}_{}.txt'.format(prefix, feat)),
                          'w', encoding="utf-8")
                     for feat in features]
    feature_light_files = [open(os.path.join(path_to_data,
                                       '{}_{}_light.txt'.format(prefix, feat)),
                          'w', encoding="utf-8")                     
                     for feat in features]
    
    with open(os.path.join(path_to_data, inp_filename), 
              encoding='utf-8') as inp_json_file:
    
        for line in tqdm_notebook(inp_json_file):
            dict_data = {}
            json_data = read_json_line(line)
            for i in range(len(feature_files)):
                dict_data[ident] = json_data[features[i]]
                feature_files[i].write(str(dict_data))
                if (ident < 5000):
                    feature_light_files[i].write(str(dict_data))
            ident += 1
    for i in feature_light_files:
        i.close()
    for i in feature_files:
        i.close()

In [39]:
PATH_TO_DATA = 'C:\mlcourse\Databases\kaggle_medium'

In [40]:
extract_features_and_write(PATH_TO_DATA, 'train.json', is_train=True)

In [41]:
extract_features_and_write(PATH_TO_DATA, 'test.json', is_train=False)

Add the following groups of features:

- Tf-Idf with article content (ngram_range=(1, 2), max_features=100000 but you can try adding more)
- Tf-Idf with article titles (ngram_range=(1, 2), max_features=100000 but you can try adding more)
- Time features: publication hour, whether it's morning, day, night, whether it's a weekend
- Bag of authors (i.e. One-Hot-Encoded author names)

In [42]:
prefix = 'train'
features = ['content', 'published', 'title', 'author']

light_files = [os.path.join(PATH_TO_DATA,
                                       '{}_{}_light.txt'.format(prefix, feat))                                         
                     for feat in features]

In [43]:
dict_data = {}
#for i in range(len(light_files)):
for i in range(1):
    with open(light_files[i], "r", encoding='utf-8') as inp_file:
        for line in inp_file:
            print(line)
        #for line in tqdm_notebook(inp_file):
        #    print(line)
            #data.append(read_json_line(line))
        #dict_data[features[i]] = data

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [44]:
for i in range(1):
    with open(light_files[i]) as file:
        for line in file:
            print(line)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 609: character maps to <undefined>

In [ ]:
import codecs

df = pd.DataFrame({})
#for k in range(len(features[1:2])):
for k in range(1):
    array = ''
    f = codecs.open(light_files[2], "r", "utf-8")
    for file in f:
        array += file
        arr = array.split("}{")
    arr[0] = arr[0][1:]
    arr[-1] = arr[-1][:-1]
    for i in range(len(arr)):
        arr[i] = "{" + arr[i] + "}"
    data = []
    for i in range(len(arr)):
        data.append(list(eval(arr[i]).values())[0])
    ser_buf = pd.Series(data)
    df = ser_buf.join()

In [248]:
array = ''
f = codecs.open(light_files[2], "r", "utf-8")
for file in f:
    array += file
    arr = array.split("}{")

In [249]:
arr[0] = arr[0][1:]
arr[-1] = arr[-1][:-1]
for i in range(len(arr)):
    arr[i] = "{" + arr[i] + "}"
data = []


In [250]:
for i in range(len(arr)):
    data.append(list(eval(arr[i]).values())[0])
ser_buf = pd.Series(data)
df = ser_buf.join()

AttributeError: 'Series' object has no attribute 'join'

In [242]:
df = ser_buf.join()

""


In [65]:
with open(light_files[]) as file:
    array = [row.strip() for row in file]
arr = array[0].split("}{")
arr[0] = arr[0][1:]
arr[-1] = arr[-1][:-1]
for i in range(len(arr)):
    arr[i] = "{" + arr[i] + "}"

In [104]:
df = pd.Series()
data = []
for i in range(len(arr)):
    data.append(eval(arr[i]).values())

In [190]:
with open(light_files[3]) as file:
    array = [row.strip() for row in file]

In [191]:
array

["{0: {'name': None, 'url': 'https://medium.com/@Medium', 'twitter': '@Medium'}}{1: {'name': None, 'url': 'https://medium.com/@Medium', 'twitter': '@Medium'}}{2: {'name': None, 'url': 'https://medium.com/@aelcenganda', 'twitter': '@aelcenganda'}}{3: {'name': None, 'url': 'https://medium.com/@vaibhavkhulbe', 'twitter': '@vaibhav_khulbe'}}{4: {'name': None, 'url': 'https://medium.com/@vaibhavkhulbe', 'twitter': '@vaibhav_khulbe'}}{5: {'name': None, 'url': 'https://medium.com/@pettykate', 'twitter': '@PettyKate'}}{6: {'name': None, 'url': 'https://medium.com/@exedre', 'twitter': '@exedre'}}{7: {'name': None, 'url': 'https://medium.com/@RaghavHaran', 'twitter': '@RaghavHaran'}}{8: {'name': None, 'url': 'https://medium.com/@___fl', 'twitter': '@___fl'}}{9: {'name': None, 'url': 'https://medium.com/@RaghavHaran', 'twitter': '@RaghavHaran'}}{10: {'name': None, 'url': 'https://medium.com/@1000ant', 'twitter': '@1000ant'}}{11: {'name': None, 'url': 'https://medium.com/@onemonth', 'twitter': Non

In [230]:
#with open(light_files[2]) as file:
#    array = [row.strip() for row in file]
    
    
import codecs
array = ''
f = codecs.open(light_files[2], "r", "utf-8")
for file in f:
    array += file

In [231]:
arr = array.split("}{")
arr[0] = arr[0][1:]
arr[-1] = arr[-1][:-1]
for i in range(len(arr)):
    arr[i] = "{" + arr[i] + "}"

In [239]:
df = pd.Series()
data = []
for i in range(len(arr)):
    data.append(list(eval(arr[i]).values())[0])

In [244]:
data

['Medium Terms of Service – Medium Policy – Medium',
 'Amendment to Medium Terms of Service Applicable to U.S. Government Users',
 '走入山與海之間：閩東大刀會和兩岸走私 – Yun-Chen Chien（簡韻真） – Medium',
 'How fast can a camera get? – What comes to mind – Medium',
 'A game for the lonely fox – What comes to mind – Medium',
 'Now That’s Comedy – Kate Reed Petty – Medium',
 'Che fine ha fatto «Gola Profonda»? – exedre – Medium',
 'Airbnb Guerilla Usability Testing – Interactive Mind – Medium',
 'A Guerilla Usability Test on Dropbox Photos – Design Insights from Designer Fund Bridge – Medium',
 'How to Get Any Job You Want (even if you’re unqualified)',
 'On Trump, Brexit and Predictions; – E² – Medium',
 '29 Books Every Entrepreneur Should Read in 2016 – Every 30 Days – Medium',
 'The Basics of Staking in Poker – Hacker Noon',
 'Crônicas do Mundial (29/09): Voltamos – Gregório Jung – Medium',
 'Why Invest in Esports? Common Questions about Esports: Answered',
 'Want a Better Pitch? Watch This. – Firm Narrat

In [245]:
pd.Series(data)

0        Medium Terms of Service – Medium Policy – Medium
1       Amendment to Medium Terms of Service Applicabl...
2       走入山與海之間：閩東大刀會和兩岸走私 – Yun-Chen Chien（簡韻真） – Medium
3       How fast can a camera get? – What comes to min...
4       A game for the lonely fox – What comes to mind...
5            Now That’s Comedy – Kate Reed Petty – Medium
6       Che fine ha fatto «Gola Profonda»? – exedre – ...
7       Airbnb Guerilla Usability Testing – Interactiv...
8       A Guerilla Usability Test on Dropbox Photos – ...
9       How to Get Any Job You Want (even if you’re un...
10        On Trump, Brexit and Predictions; – E² – Medium
11      29 Books Every Entrepreneur Should Read in 201...
12           The Basics of Staking in Poker – Hacker Noon
13      Crônicas do Mundial (29/09): Voltamos – Gregór...
14      Why Invest in Esports? Common Questions about ...
15      Want a Better Pitch? Watch This. – Firm Narrat...
16      The Birth of a Venture Capital Investment Thes...
17      Why Sa

In [ ]:
names = []
urls = []
twitters = []

for i in range(len(data)):
    names.append(list(data[1])[0]['name'])
    urls.append(list(data[1])[0]['url'])
    twitters.append(list(data[1])[0]['twitter'])
df = pd.DataFrame({'name' : names, 'url': urls, 'twitter': twitters})
